In [2]:
import numpy as np
import pandas as pd
from astropy.io import fits
import glob
from collections import defaultdict
import sqlite3 as lite

# Read in the field information
#root = "/home/esmee/Documents/DBDM/DDM2017-master/FinalProject/" 
root = "/disks/strw9/stoop/DBDM/DDM2017-master/FinalProject/Final-Project-DBDM/"

In [170]:
field_info = pd.read_csv(root + 'file_info_for_problem.csv')
field_info.insert(column="StarID", loc=4, value = 0)

# Make the filters of the same form as for StarTable
for field in range(1,4):
    #Find the index of filter Ks
    index_ks = np.where( (field_info['FieldID'] == field) & (field_info['Filter'] == 'Ks') )[0]
    sort = np.argsort(field_info['MJD'][index_ks])
    
    #Change to Ks-E00..
    tel = 1
    for i in index_ks[sort]:
        field_info['Filter'][i] = field_info['Filter'][i] + '_E00'+str(tel)
        tel +=1

/software/local/lib64/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [173]:
# Read in the different fits files
fnames = glob.glob(root + '*.fits')

#Find the number of unique stars
starIDs = []
for i, f in enumerate(fnames):
    hdu = fits.open(f)
    starIDs.append([dat for dat in hdu[1].data['StarID']])

nstars = len(np.unique(starIDs))
starCols = np.array(['ID', 'StarID', 'Ra', 'Dec', 'Flux1_Z', 'Flux1_Y', 'Flux1_J', 'Flux1_H', 'Flux1_Ks_E001',
       'Flux1_Ks_E002', 'Flux1_Ks_E003', 'Flux2_Z', 'Flux2_Y', 'Flux2_J',
       'Flux2_H', 'Flux2_Ks_E001', 'Flux2_Ks_E002', 'Flux2_Ks_E003',
       'Flux3_Z', 'Flux3_Y', 'Flux3_J', 'Flux3_H', 'Flux3_Ks_E001',
       'Flux3_Ks_E002', 'Flux3_Ks_E003', 'dFlux1_Z', 'dFlux1_Y',
       'dFlux1_J', 'dFlux1_H', 'dFlux1_Ks_E001', 'dFlux1_Ks_E002',
       'dFlux1_Ks_E003', 'dFlux2_Z', 'dFlux2_Y', 'dFlux2_J', 'dFlux2_H',
       'dFlux2_Ks_E001', 'dFlux2_Ks_E002', 'dFlux2_Ks_E003', 'dFlux3_Z',
       'dFlux3_Y', 'dFlux3_J', 'dFlux3_H', 'dFlux3_Ks_E001',
       'dFlux3_Ks_E002', 'dFlux3_Ks_E003', 'Mag1_Z', 'Mag1_Y', 'Mag1_J',
       'Mag1_H', 'Mag1_Ks_E001', 'Mag1_Ks_E002', 'Mag1_Ks_E003', 'Mag2_Z',
       'Mag2_Y', 'Mag2_J', 'Mag2_H', 'Mag2_Ks_E001', 'Mag2_Ks_E002',
       'Mag2_Ks_E003', 'Mag3_Z', 'Mag3_Y', 'Mag3_J', 'Mag3_H',
       'Mag3_Ks_E001', 'Mag3_Ks_E002', 'Mag3_Ks_E003', 'dMag1_Z',
       'dMag1_Y', 'dMag1_J', 'dMag1_H', 'dMag1_Ks_E001', 'dMag1_Ks_E002',
       'dMag1_Ks_E003', 'dMag2_Z', 'dMag2_Y', 'dMag2_J', 'dMag2_H',
       'dMag2_Ks_E001', 'dMag2_Ks_E002', 'dMag2_Ks_E003', 'dMag3_Z',
       'dMag3_Y', 'dMag3_J', 'dMag3_H', 'dMag3_Ks_E001', 'dMag3_Ks_E002',
       'dMag3_Ks_E003']) 

fmcols = ['Flux', 'dFlux', 'Mag', 'dMag']
uniq_IDs = np.unique(starIDs)
nstars = len(np.unique(starIDs))
ncols = 5 + 83
print "Table is {0} x {1}".format(nstars, ncols)
stable = np.zeros((nstars, ncols))
starTable = pd.DataFrame(data=stable, columns=starCols)

Table is 30000 x 88



# The database tables

**FieldTable**

ID | FieldID | Filename | Filter | Exp time | MJD | StarIDs |


**StarTable**

ID  | StarID  | Ra  | Dec  | Flux1_Z  | dFlux1_Z  | ..  | Flux3_Ks  | dFlux3_Ks  | Mag1_Z  | dMag1_Z  | ..  | Mag3_Ks  | dMag3_Ks  | Var_Ks |  



In [174]:

IDs_seen = []
fmcols = ['Flux', 'dFlux', 'Mag', 'dMag']

for f in fnames:
    print "Started with file {0}".format(f.replace(root, ''))
    
    hdu = fits.open(f)
    f = f.replace(root, '')
    sID = hdu[1].data['StarID']
    ind = [np.where(uniq_IDs == i)[0][0] for i in sID]
    filt = f[8:-5]
    if '-' in filt: filt = filt.replace('-','_')
    
    #Send star IDs to the field_info
    fID = f[6] #field
    name = np.sort(np.array(field_info["Filename"][(field_info["FieldID"] == int(fID)) & (field_info["Filter"] == filt)]))
    name = name[0]
    field_info["StarID"][np.where(field_info["Filename"] == name)[0][0]] = np.array_str(sID)
    
    seen = [ID in IDs_seen for ID in sID]

    # Is iD star already seen?
    if not seen[0]:
        starTable['ID'][ind] = ind
        # Input the StarID, Ra and Dec
        for c in starCols[1:4]:
            starTable[c][ind] = hdu[1].data[c]

        [IDs_seen.append(ID) for ID in sID]
        
        # Input the flux and magnitude of this data
        for fm in fmcols:
            for t in range(1, 4):
                starTable[fm + str(t) +'_' + filt][ind] = hdu[1].data[fm + str(t)]

    # Go through all other flux, mag and filters
    else:
        for fm in fmcols:
            for t in range(1, 4):
                starTable[fm + str(t) +'_' + filt][ind] = hdu[1].data[fm + str(t)]
    

    print "Finished"

# write tables to csv
starTable.to_csv(root+'starTable.csv')
field_info.to_csv(root+'fieldTable.csv')



Started with file Field-3-Ks-E002.fits
Ks_E002


/software/local/lib64/python2.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Finished
Started with file Field-3-Ks-E001.fits
Ks_E001
Finished
Started with file Field-2-H.fits
H
Finished
Started with file Field-1-Ks-E001.fits
Ks_E001
Finished
Started with file Field-2-Y.fits
Y
Finished
Started with file Field-3-J.fits
J
Finished
Started with file Field-1-Y.fits
Y
Finished
Started with file Field-1-Ks-E003.fits
Ks_E003
Finished
Started with file Field-1-Ks-E002.fits
Ks_E002
Finished
Started with file Field-1-H.fits
H
Finished
Started with file Field-3-H.fits
H
Finished
Started with file Field-3-Z.fits
Z
Finished
Started with file Field-2-Z.fits
Z
Finished
Started with file Field-3-Y.fits
Y
Finished
Started with file Field-1-Z.fits
Z
Finished
Started with file Field-2-Ks-E001.fits
Ks_E001
Finished
Started with file Field-2-J.fits
J
Finished
Started with file Field-1-J.fits
J
Finished


In [175]:
# Read the Tables
FieldTable = pd.read_csv(root+ "fieldTable.csv")
StarTable = pd.read_csv(root+ "starTable.csv")

FieldTable.drop('Unnamed: 0', axis=1, inplace=True)
StarTable.drop('Unnamed: 0', axis=1, inplace=True)


In [3]:
# Make the database
con = lite.connect(root+"VVV.db")
StarTable.to_sql('StarTable', con, if_exists = 'replace')
FieldTable.to_sql('FieldTable', con, if_exists = 'replace')

NameError: name 'StarTable' is not defined

# The Queries

**R1:** Find all images observed between MJD = 56800 and MJD = 57300 and give me the number of stars detected with S/N > 5 in each image.

**R2:** Find the objects that have J-H > 1.5

**R3:** Find the objects where Ks differs by more than 20 times the flux uncertainty from the mean flux.

**R4:** Find all catalogues that exist for a given field.

**R5:** For a given field I would like to retrieve the Y,Z,J,H and Ks magnitudes for all stars with S/N > 30 in Y, Z, J, H and Ks. 




In [4]:
# R1


con = lite.connect(root+'VVV.db')
R1command = "SELECT Filename, MJD, StarID, Filter FROM FieldTable WHERE MJD BETWEEN 56800 AND 57300"
t_images = pd.read_sql(R1command, con)

for i, stars in enumerate(t_images['StarID']):
    # Need filter to get the right fluxes
    filt = t_images['Filter'][i]
    
    #Get the first and last starID
    sIDs = stars[1:-1].split(' ')
    sIDs = np.array([s for s in sIDs if s!= ''])
    #print "Found stars {0} - {1}".format(sIDs[0], sIDs[-1])
    
    #Get the right flux belonging to filt and stars with stars ID between sIDs[0], sIDs[-1]
    R1command2 = "SELECT ID, Flux1_{0}, dFlux1_{0} FROM StarTable\
                WHERE ID BETWEEN {1} AND {2}".format(str(filt), sIDs[0], sIDs[-1])
    
    #print R1command2
    fluxes = pd.read_sql(R1command2, con)
    
    # Find the stars with S/N > 5. Define S/N as Flux/dFlux
    result = fluxes['ID'][np.where(fluxes['Flux1_'+filt]/fluxes['dFlux1_'+filt] > 5.0)[0]]
    if len(result > 0): print "\nFound {0} stars in image in file {1}".format(len(result), t_images['Filename'][i])
    


Found 9862 stars in image in file Z-ADP.2017-01-18T11:58:36.905.fits

Found 9985 stars in image in file J-ADP.2017-01-18T11:58:35.781.fits

Found 9990 stars in image in file H-ADP.2017-01-18T11:58:35.780.fits

Found 9992 stars in image in file Ks-ADP.2016-05-25T15:33:43.377.fits

Found 9967 stars in image in file Y-ADP.2017-01-18T11:58:36.901.fits


In [10]:
# R2

R2command = """SELECT StarID from StarTable WHERE Flux1_J - Flux1_H + dFlux1_J + dFlux1_H > 1.5 AND Flux1_J - Flux1_H - dFlux1_J - dFlux1_H > 1.5"""
stars = pd.read_sql(R2command, con)
print "Found {} stars with StarID's".format(len(stars))
print stars.astype(int)

Found 1228 stars with StarID's
      StarID
0         97
1        189
2        299
3        329
4        472
5        545
6        549
7        633
8        665
9        714
10       737
11       752
12       761
13       782
14       804
15       848
16       867
17       875
18       917
19       945
20       947
21       970
22       989
23      1035
24      1070
25      1096
26      1100
27      1141
28      1144
29      1152
...      ...
1198  309328
1199  309378
1200  309460
1201  309464
1202  309471
1203  309521
1204  309524
1205  309527
1206  309544
1207  309591
1208  309597
1209  309600
1210  309604
1211  309632
1212  309644
1213  309645
1214  309735
1215  309758
1216  309788
1217  309814
1218  309860
1219  309879
1220  309880
1221  309883
1222  309902
1223  309916
1224  309937
1225  309956
1226  309971
1227  309999

[1228 rows x 1 columns]


In [ ]:
# R3

# What is ment with mean flux uncertainty?